In [42]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

from time import sleep
import tqdm
import json
import time
import re

import requests
from bs4 import BeautifulSoup
from html2text import html2text

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings('ignore')

import numpy as np


In [43]:
category_links = [
    ('https://www.nn.ru/text/?rubric=economics&page=', 'economics'),
    ('https://www.nn.ru/text/?rubric=gorod&page=', 'gorod'),
    ('https://www.nn.ru/text/?rubric=culture&page=', 'culture'),
    ('https://www.nn.ru/text/?rubric=incidents&page=', 'incidents'),
    ('https://www.nn.ru/text/?rubric=politics&page=', 'politics'),
    ('https://www.nn.ru/text/?rubric=world&page=', 'world')
]

'''
sport - 300
economics - 935
gorod - много
science - 146
culture - много
incidents - много
politics - 1500
world - много
'''

PAGES_LOAD = 25

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


chrome_options = Options()
prefs = {"profile.managed_default_content_settings.images": 2,
            "profile.default_content_settings.cookies": 2,
            "profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)
chrome_options.add_argument('--disable-application-cache')

driver = webdriver.Chrome(options=chrome_options)


def sleeepy():
    """Not to be banned by robot check"""
    ...
    sleep(np.random.randint(0, 50)/10)


category_res = {}
for link, category in category_links:
    news_links = []
    for page_id in range(1, PAGES_LOAD):

        if page_id % 2 == 0: sleeepy()

        page_link = link + str(page_id)
        driver.get(page_link)
        page_news = driver.find_elements(By.CLASS_NAME, "h9Jmx")
        
        if len(page_news) == 0: break
        for element in page_news:
            news_links.append(element.find_element(By.TAG_NAME, 'a').get_attribute('href'))
        print(page_link, len(news_links))
    category_res[category] = news_links

    sleeepy()
driver.quit()

data = json.dumps(category_res)
with open("cache/links.json", "w") as f:
    f.write(data)

In [45]:
with open("cache/links.json", "r") as f:
    links = json.load(f)

leng = []
total = 0
for values in list(links.values()):
    len_values = len(values)
    leng.append(len_values)
    total += len_values
print(leng, total)

[937, 960, 960, 960, 960, 960] 5737


In [ ]:
filename = "cache/news.json"

news = []
driver = webdriver.Chrome(chrome_options=chrome_options)
for category in links:
    for link in tqdm.tqdm(links[category], desc=category.upper()):
        try:
            driver.get(link)
            page_text_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'qQq9J')]")
            raw_text = " ".join([i.get_property('innerText') for i in page_text_elements]).replace('\n', " ")
            text = re.sub(" +", " ", raw_text)
            key_words = driver.find_element(By.XPATH, "//meta[contains(@name, 'news_keywords')]").get_attribute('content')
            title = driver.find_element(By.XPATH, "//h1[contains(@itemprop, 'headline')]").get_property('innerText')
            news.append({
                    'article_id' : link,
                    'title': title,
                    'category': category,
                    'tags': key_words,
                    'text': text
                })
        except:
            print('Error')
            continue
driver.quit()

with open(filename, "w", encoding='UTF-8') as f:
    f.write(json.dumps({
        'catalog': news
    }, ensure_ascii=False, indent=4))

In [ ]:
with open("cache/news.json", 'r', encoding="UTF-8") as f:
    news = pd.DataFrame(json.load(f).get('catalog'))
news

,article_id,title,category,tags,text
0,https://www.nn.ru/text/economics/2023/01/17/71...,"Пенсионер показал, как выросли его коммунальны...",economics,"Квитанции,Пенсия,Платежки,Рост цен,Субсидии,ЖК...",Пока одни россияне молча отходят от шока после...
1,https://www.nn.ru/text/economics/2023/01/17/71...,Евро по 90? Эксперты — возможен ли такой сцена...,economics,"Центробанк,Евро,Курс валют,Курс евро,Нефть,Эко...","Россияне начали покупать евро, веря в прогнозы..."
2,https://www.nn.ru/text/economics/2023/01/14/71...,"«Я набрала кредитов на 2,7 миллиона»: исповедь...",economics,"Биполярное расстройство,Кредиты,Психические за...",Я кредитный маньяк. И у меня биполярное расстр...
3,https://www.nn.ru/text/economics/2023/01/13/71...,Назад в прошлое! В российские магазины вернули...,economics,"Банк России,Центробанк,Пятирублевая банкнота,5...",В России стали возвращаться в оборот пятирубле...
4,https://www.nn.ru/text/economics/2023/01/13/71...,Мужчина два года судился с налоговой за перепл...,economics,"ВС РФ,Верховный суд РФ,ФНС,Деньги,Налоги,Налог...",Формально в России непрецедентное право: схожи...
...,...,...,...,...,...
1231,https://www.nn.ru/text/gorod/2022/11/05/71793827/,30 нижегородских ДУКов могут лишить лицензии и...,gorod,"Нижний Новгород,ДУК,Нижегородская область,ЖКХ,...",Тридцати домоуправляющим компаниям Нижегородск...
1232,https://www.nn.ru/text/gorod/2022/11/05/71790353/,"«Устал бегать туда-сюда»: история айтишника, к...",gorod,"Германия,Грузия,Спецоперация,Эмиграция,Частичн...",Москвич Максим Кокорев покинул страну после на...
1233,https://www.nn.ru/text/gorod/2022/11/05/71793392/,Инсталляция на кремле и концерт «Любэ». Как от...,gorod,"4 ноября,Нижний Новгород,День народного единст...",Вчера в России отмечали День народного единств...
1234,https://www.nn.ru/text/gorod/2022/11/05/71793224/,В опустевшем торговом центре «Шоколад» снова о...,gorod,"Нижний Новгород,Белинского,ТЦ Шоколад,Кинотеат...",Кинотеатр на шесть залов планируют открыть в т...
